In [1]:
import numpy as np

In [2]:
from miniccpy.driver import run_scf, run_cc_calc

basis = '6-31g'
nfrozen = 0

atom = [['He', (0.000, 0.000, 0.000)]] 

fock, g, e_hf, o, v = run_scf(atom, basis, nfrozen)

T, E_corr = run_cc_calc(fock, g, o, v, method='ccsd', convergence=1.0e-012, diis_size=100)

converged SCF energy = -2.85516042615445
   System Information:
   ----------------------------------------------------
     Number of correlated electrons = 2
     Number of correlated orbitals = 4
     Number of frozen orbitals = 0
     Number of occupied orbitals = 2
     Number of unoccupied orbitals = 2
     Charge = 0
     Symmetry group = SO3
     Spin multiplicity of reference = 1

      MO #        Energy (a.u.)      Symmetry    Occupation
  -----------------------------------------------------------
         1            -0.914127           s+0           2.0
         2             1.399859           s+0           0.0

   Nuclear Repulsion Energy = 0
   Reference Energy = -2.8551604261544465

    ==> CCSD amplitude equations <==

     Iter               Energy                 |dE|                 |dT|
        0      -0.011200122910       0.011200122910       0.455340990536    0.00m 0.01s
        1      -0.014037837610       0.002837714700       0.119622293820    0.00m 0.01s
  

In [3]:
print(T[0])

[[0.00208456 0.        ]
 [0.         0.00208456]]


In [4]:
print(T[1])

[[[[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.         -0.06589656]
   [ 0.06589656  0.        ]]]


 [[[ 0.          0.06589656]
   [-0.06589656  0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]]]


In [5]:
fock

array([[-9.14126629e-01,  0.00000000e+00,  1.78546067e-12,
         0.00000000e+00],
       [ 0.00000000e+00, -9.14126629e-01,  0.00000000e+00,
         1.78546067e-12],
       [ 1.78579374e-12,  0.00000000e+00,  1.39985934e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  1.78579374e-12,  0.00000000e+00,
         1.39985934e+00]])

In [6]:
VT13 = -np.einsum("mnef,fn,ei,am->ai", g[o, o, v, v], T[0], T[0], T[0], optimize=True)

In [7]:
VT13

array([[-2.06228958e-09, -0.00000000e+00],
       [-0.00000000e+00, -2.06228958e-09]])

In [8]:
x1 = -g[0, 1, 2, 3] * T[0][1, 1] * T[0][0, 0] * T[0][0, 0] # -<12||34> * <4|t1|2> * <3|t1|1> * <3|t1|1>
x2 = -g[0, 1, 3, 2] * T[0][0, 1] * T[0][1, 0] * T[0][0, 0] # -<12||43> * <3|t1|2> * <4|t1|1> * <3|t1|1> (SF)
x3 = -g[1, 0, 2, 3] * T[0][1, 0] * T[0][0, 0] * T[0][0, 1] # -<21||34> * <4|t1|1> * <3|t1|1> * <3|t1|2> (SF)
x4 = -g[1, 0, 3, 2] * T[0][0, 0] * T[0][1, 0] * T[0][0, 1] # -<21||43> * <3|t1|1> * <4|t1|1> * <3|t1|2> (SF)

print(x1,x2,x3,x4)

-2.062289583104001e-09 0.0 0.0 -0.0


In [9]:
VT1T2 = (
            np.einsum("mnef,fn,aeim->ai", g[o, o, v, v], T[0], T[1], optimize=True)
            - 0.5 * np.einsum("mnef,am,efin->ai", g[o, o, v, v], T[0], T[1], optimize=True)
            - 0.5 * np.einsum("mnef,ei,afmn->ai", g[o, o, v, v], T[0], T[1], optimize=True)
)

In [10]:
VT1T2

array([[3.12740253e-05, 0.00000000e+00],
       [0.00000000e+00, 3.12740253e-05]])

# Huckel-like Models

In [27]:
def get_linear_huckel(n):
    alpha = 0.0
    beta = -1.0
    
    h1 = np.diag(alpha*np.ones(n)) + np.diag(beta*np.ones(n-1), -1) + np.diag(beta*np.ones(n-1), 1)
    h2 = np.zeros((n, n, n, n))
    for i in range(n):
        h2[i, i, i, i] = 2.0 * abs(beta)
        
    return h1, h2

def huckel_system(h1, h2):
    
    from miniccpy.integrals import spatial_to_spinorb, get_fock
    from miniccpy.energy import hf_energy, hf_energy_from_fock
    from miniccpy.printing import print_custom_system_information
    
    norbitals = h1.shape[0]
    nelectron = h1.shape[0]

    # Diagonalize one-body matrix to get Huckel eigenstates as the single-particle spatial orbital (MO) basis
    mo_energy, mo_coeff = np.linalg.eigh(h1)

    # Perform AO to MO transformation in spatial orbital basis 
    e1int = np.einsum("ip,ij,jq->pq", mo_coeff, h1, mo_coeff, optimize=True)
    e2int = np.einsum("ip,jq,kr,ls,ijkl->pqrs", mo_coeff, mo_coeff, mo_coeff, mo_coeff, h2, optimize=True)
    # Convert from spatial orbitals to spin-orbitals
    z, g = spatial_to_spinorb(e1int, e2int)
    # Antisymmetrize two-body integrals
    g -= np.transpose(g, (0, 1, 3, 2))

    # Get correlated slicing arrays
    o = slice(0, nelectron)
    v = slice(nelectron, 2 * norbitals)

    # build Fock matrix and HF energy
    fock = get_fock(z, g, o)
    e_hf = hf_energy(z, g, o)
    e_hf_test = hf_energy_from_fock(fock, g, o)
    assert(abs(e_hf - e_hf_test) < 1.0e-09)

    # Print system information
    print_custom_system_information(fock, nelectron, 0, e_hf)
    
    return fock, g, o, v

In [28]:
# Huckel model for ethylene
h1, h2 = get_linear_huckel(2)

# Get MO spin-orbital integrals from Huckel parameterization
fock, g, o, v = huckel_system(h1, h2)

# Run CC calculation (for 2-site model, DIIS is singular)
T, E_corr = run_cc_calc(fock, g, o, v, method='ccsd', convergence=1.0e-012, diis_size=100)

   System Information (Custom):
   ----------------------------------------------------
     Number of correlated electrons = 2
     Number of correlated orbitals = 4
     Number of frozen orbitals = 0
     Number of occupied orbitals = 2
     Number of unoccupied orbitals = 2
      MO #        Energy (a.u.)      Symmetry    Occupation
  -----------------------------------------------------------
     1(1𝜶)            -0.000000            C1           1.0
     2(1𝜷)            -0.000000            C1           1.0
     3(2𝜶)             2.000000            C1           0.0
     4(2𝜷)             2.000000            C1           0.0

   Reference Energy = -0.9999999999999999

    ==> CCSD amplitude equations <==

     Iter               Energy                 |dE|                 |dT|
        0      -0.250000000000       0.250000000000       2.000000000000    0.00m 0.01s
        1      -0.234375000000       0.015625000000       0.125000000000    0.00m 0.01s
        2      -0.23626708984

In [29]:
# Huckel model for butadiene
h1, h2 = get_linear_huckel(4)

# Get MO spin-orbital integrals from Huckel parameterization
fock, g, o, v = huckel_system(h1, h2)

# Run CC calculation
T, E_corr = run_cc_calc(fock, g, o, v, method='ccsd', convergence=1.0e-012, diis_size=6)

   System Information (Custom):
   ----------------------------------------------------
     Number of correlated electrons = 4
     Number of correlated orbitals = 8
     Number of frozen orbitals = 0
     Number of occupied orbitals = 4
     Number of unoccupied orbitals = 4
      MO #        Energy (a.u.)      Symmetry    Occupation
  -----------------------------------------------------------
     1(1𝜶)            -0.618034            C1           1.0
     2(1𝜷)            -0.618034            C1           1.0
     3(2𝜶)             0.381966            C1           1.0
     4(2𝜷)             0.381966            C1           1.0
     5(3𝜶)             1.618034            C1           0.0
     6(3𝜷)             1.618034            C1           0.0
     7(4𝜶)             2.618034            C1           0.0
     8(4𝜷)             2.618034            C1           0.0

   Reference Energy = -2.47213595499958

    ==> CCSD amplitude equations <==

     Iter               Energy          

In [30]:
# Huckel model for butadiene
h1, h2 = get_linear_huckel(12)

# Get MO spin-orbital integrals from Huckel parameterization
fock, g, o, v = huckel_system(h1, h2)

# Run CC calculation
T, E_corr = run_cc_calc(fock, g, o, v, method='ccsd', convergence=1.0e-012, diis_size=6)

   System Information (Custom):
   ----------------------------------------------------
     Number of correlated electrons = 12
     Number of correlated orbitals = 24
     Number of frozen orbitals = 0
     Number of occupied orbitals = 12
     Number of unoccupied orbitals = 12
      MO #        Energy (a.u.)      Symmetry    Occupation
  -----------------------------------------------------------
     1(1𝜶)            -0.941884            C1           1.0
     2(1𝜷)            -0.941884            C1           1.0
     3(2𝜶)            -0.770912            C1           1.0
     4(2𝜷)            -0.770912            C1           1.0
     5(3𝜶)            -0.497021            C1           1.0
     6(3𝜷)            -0.497021            C1           1.0
     7(4𝜶)            -0.136129            C1           1.0
     8(4𝜷)            -0.136129            C1           1.0
     9(5𝜶)             0.290790            C1           1.0
    10(5𝜷)             0.290790            C1           